# Process Files from CSV Mapping

This notebook loads a filename mapping CSV (created by `renaming_file_template.ipynb`) and processes files to Cloud Optimized GeoTIFFs (COGs) with the new filenames.

## Features
- **Load CSV mapping** - Import pre-defined filename transformations
- **Preview before processing** - Review what will be processed
- **Batch COG conversion** - Convert all files to COGs
- **Track results** - Save processing results to CSV

## Workflow
1. Generate mapping CSV using `renaming_file_template.ipynb`
2. Review and validate the CSV
3. Run this notebook to process files

---

## 📋 Step 1: Basic Configuration

Set your event name to load the corresponding CSV mapping file:

In [3]:
# ========================================
# INPUTS
# ========================================

# S3 Configuration
BUCKET = 'nasa-disasters'    # S3 bucket (DO NOT CHANGE)

# Event Details
EVENT_NAME = '202510_Flood_AK'  # Must match the event name used in renaming_file_template.ipynb
SUB_PRODUCT_NAME = 'sentinel2'

# CSV Mapping Configuration
CSV_DIR = 'file-mapping'  # Directory where CSV mappings are stored
CSV_FILENAME = f'{EVENT_NAME}-{SUB_PRODUCT_NAME}.csv'  # CSV filename (usually {EVENT_NAME}-{subproductName}.csv)

# Processing Options
CHECK_SOURCE_IS_COG = True  # Check if source files are already valid COGs
SKIP_IF_SOURCE_IS_COG = True  # Skip processing if source is already a valid COG
OVERWRITE = False      # Set to True to replace existing files in S3 (after converting to COG)
VERIFY = True          # Set to True to verify COGs after creation
SAVE_RESULTS = True    # Save processing results to CSV

# Output
OUTPUT_DIR = 'csv_stats'  # Directory for results CSV


## 📂 Step 2: Load CSV Mapping

Load the filename mapping CSV created by the renaming template:

In [ ]:
import pandas as pd
from pathlib import Path

# Construct CSV path
csv_path = Path(CSV_DIR) / CSV_FILENAME

print("📂 LOADING CSV MAPPING")
print("="*80)
print(f"\nLooking for: {csv_path}")

# Check if CSV exists
if not csv_path.exists():
    print(f"\n❌ ERROR: CSV file not found!")
    print(f"\nMake sure you've run 'renaming_file_template.ipynb' first to create the mapping CSV.")
    print(f"Expected location: {csv_path.absolute()}")
    mapping_df = pd.DataFrame()
else:
    # Load CSV
    mapping_df = pd.read_csv(csv_path)
    
    print(f"\n✅ Successfully loaded CSV")
    print(f"\n📊 Mapping details:")
    print(f"   Total entries: {len(mapping_df)}")
    
    if 'status' in mapping_df.columns:
        print(f"   Valid files:   {len(mapping_df[mapping_df['status'] == 'valid'])}")
        print(f"   Invalid files: {len(mapping_df[mapping_df['status'] == 'invalid'])}")
    
    if 'file_size_gb' in mapping_df.columns:
        print(f"   Total size:    {mapping_df['file_size_gb'].sum():.2f} GB")
    
    if 'category' in mapping_df.columns:
        print(f"\n📂 By category:")
        category_counts = mapping_df['category'].value_counts()
        for category, count in category_counts.items():
            print(f"   • {category}: {count} files")
    
    print(f"\n✅ Ready to filter and process files")
    print("\n" + "="*80)
    
    # Display first few rows
    print("\n📋 Preview (first 5 rows):")
    print(mapping_df.head())

## 🔍 Step 3: Filter Files to Process

Optionally filter which files to process (by category, size, etc.):

In [5]:
if not mapping_df.empty:
    # Filter out uncategorized files (they won't be processed)
    files_to_process = mapping_df[mapping_df['status'] == 'valid'].copy()
    
    # Optional: Filter by category
    # Uncomment and modify to process only specific categories:
    # CATEGORIES_TO_PROCESS = ['trueColor', 'colorInfrared']
    # files_to_process = files_to_process[files_to_process['category'].isin(CATEGORIES_TO_PROCESS)]
    
    # Optional: Filter by file size
    # Uncomment to process only files smaller than a certain size:
    # MAX_SIZE_GB = 5.0
    # files_to_process = files_to_process[files_to_process['file_size_gb'] <= MAX_SIZE_GB]
    
    print("📊 FILES TO PROCESS")
    print("="*80)
    print(f"\nTotal files: {len(files_to_process)}")
    print(f"Total size:  {files_to_process['file_size_gb'].sum():.2f} GB")
    
    if len(files_to_process) > 0:
        print(f"\n📂 By category:")
        category_counts = files_to_process['category'].value_counts()
        for category, count in category_counts.items():
            print(f"   • {category}: {count} files")
        
        print(f"\n✅ Ready to process {len(files_to_process)} files")
    else:
        print("\n⚠️  No files match the filter criteria")
    
    print("\n" + "="*80)
else:
    print("⚠️ No mapping data loaded. Check Step 2.")
    files_to_process = pd.DataFrame()

NameError: name 'mapping_df' is not defined

## 🌐 Step 4: Connect to S3

Initialize S3 client for downloading source files and uploading processed COGs:

In [ ]:
from core.s3_operations import initialize_s3_client

print("🌐 Connecting to S3...")
s3_client, fs = initialize_s3_client(bucket_name=BUCKET, verbose=True)

if s3_client:
    print("\n✅ Successfully connected to S3")
    print("   Ready to download source files and upload processed COGs")
else:
    print("\n❌ Failed to connect to S3")
    print("   Check your AWS credentials and permissions")

## ⚙️ Step 5: Process Files to COGs

Convert files to Cloud Optimized GeoTIFFs with the new filenames:

## 🔍 Step 3.5: Validate Source Files (Check if Already COGs)

Check if source files are already valid COGs to avoid unnecessary processing:

In [ ]:
if not files_to_process.empty and CHECK_SOURCE_IS_COG and s3_client:
    from core.validation import is_s3_file_cog
    
    print("🔍 VALIDATING SOURCE FILES")
    print("="*80)
    print(f"\nChecking if {len(files_to_process)} source files are already valid COGs...")
    print("This may take a while as files need to be downloaded for validation.\n")
    
    # Add COG status columns
    files_to_process['source_is_cog'] = False
    files_to_process['source_cog_details'] = None
    
    # Check each file
    already_cog_count = 0
    not_cog_count = 0
    error_count = 0
    
    for idx, row in files_to_process.iterrows():
        source_path = row['original_s3_path']
        filename = row['original_filename']
        
        print(f"[{idx+1}/{len(files_to_process)}] Checking: {filename}...", end=" ")
        
        try:
            is_cog, details = is_s3_file_cog(s3_client, BUCKET, source_path, verbose=False)
            
            files_to_process.at[idx, 'source_is_cog'] = is_cog
            files_to_process.at[idx, 'source_cog_details'] = str(details)
            
            if is_cog:
                print("✅ Already a COG")
                already_cog_count += 1
            else:
                print("❌ Not a COG")
                not_cog_count += 1
        except Exception as e:
            print(f"⚠️  Error: {str(e)}")
            error_count += 1
            files_to_process.at[idx, 'source_is_cog'] = False
            files_to_process.at[idx, 'source_cog_details'] = f"Error: {str(e)}"
    
    print("\n" + "="*80)
    print(f"\n📊 VALIDATION RESULTS:")
    print(f"   ✅ Already valid COGs: {already_cog_count}")
    print(f"   ❌ Not COGs (need processing): {not_cog_count}")
    print(f"   ⚠️  Errors: {error_count}")
    
    # Filter out files that are already COGs if SKIP_IF_SOURCE_IS_COG is True
    if SKIP_IF_SOURCE_IS_COG:
        files_before = len(files_to_process)
        files_to_process = files_to_process[files_to_process['source_is_cog'] == False].copy()
        files_after = len(files_to_process)
        
        skipped = files_before - files_after
        if skipped > 0:
            print(f"\n⏭️  SKIPPING {skipped} files that are already valid COGs")
            print(f"   Files remaining to process: {files_after}")
    
    print("\n" + "="*80)
    
elif not CHECK_SOURCE_IS_COG:
    print("ℹ️  Source COG validation disabled (CHECK_SOURCE_IS_COG = False)")
elif files_to_process.empty:
    print("⚠️  No files to validate")
else:
    print("⚠️  S3 client not connected, skipping validation")

In [ ]:
if not files_to_process.empty and s3_client:
    from core.cog_processing import process_single_file
    from core.s3_operations import check_s3_file_exists
    import time
    
    print("🚀 STARTING COG PROCESSING")
    print("="*80)
    print(f"\nProcessing {len(files_to_process)} files...")
    print("This may take several minutes depending on file sizes.\n")
    
    # Track results
    results = []
    
    for idx, row in files_to_process.iterrows():
        start_time = time.time()
        
        source_path = row['original_s3_path']
        dest_path = row['output_s3_path']
        category = row['category']
        
        print(f"\n[{idx+1}/{len(files_to_process)}] Processing: {row['original_filename']}")
        print(f"    Category: {category}")
        print(f"    Size: {row['file_size_gb']:.2f} GB")
        print(f"    Output: {row['new_filename']}")
        
        # Check if file already exists (unless OVERWRITE is True)
        if not OVERWRITE:
            if check_s3_file_exists(s3_client, BUCKET, dest_path):
                print(f"    ⏭️  SKIPPED (already exists)")
                results.append({
                    'source_file': row['original_filename'],
                    'output_file': row['new_filename'],
                    'category': category,
                    'status': 'skipped',
                    'time_seconds': 0,
                    'error': 'File already exists'
                })
                continue
        
        try:
            # Process file to COG
            success = process_single_file(
                s3_client=s3_client,
                bucket=BUCKET,
                source_key=source_path,
                dest_key=dest_path,
                nodata=None,  # Auto-detect nodata value
                verify=VERIFY
            )
            
            elapsed = time.time() - start_time
            
            if success:
                print(f"    ✅ SUCCESS ({elapsed:.1f}s)")
                results.append({
                    'source_file': row['original_filename'],
                    'output_file': row['new_filename'],
                    'category': category,
                    'status': 'success',
                    'time_seconds': elapsed,
                    'error': None
                })
            else:
                print(f"    ❌ FAILED ({elapsed:.1f}s)")
                results.append({
                    'source_file': row['original_filename'],
                    'output_file': row['new_filename'],
                    'category': category,
                    'status': 'failed',
                    'time_seconds': elapsed,
                    'error': 'Processing failed'
                })
        
        except Exception as e:
            elapsed = time.time() - start_time
            print(f"    ❌ ERROR: {str(e)}")
            results.append({
                'source_file': row['original_filename'],
                'output_file': row['new_filename'],
                'category': category,
                'status': 'failed',
                'time_seconds': elapsed,
                'error': str(e)
            })
    
    # Create results DataFrame
    results_df = pd.DataFrame(results)
    
    print("\n" + "="*80)
    print("\n🎉 PROCESSING COMPLETE!")
    
else:
    print("⚠️ Cannot process: No files to process or S3 not connected")
    results_df = pd.DataFrame()

## 📊 Step 6: View Results

Display processing statistics and results:

In [ ]:
if not results_df.empty:
    print("📊 PROCESSING STATISTICS")
    print("="*80)
    
    # Overall stats
    total = len(results_df)
    success = len(results_df[results_df['status'] == 'success'])
    failed = len(results_df[results_df['status'] == 'failed'])
    skipped = len(results_df[results_df['status'] == 'skipped'])
    
    print(f"\n🔢 Overall:")
    print(f"   Total files:   {total}")
    print(f"   ✅ Success:    {success} ({success/total*100:.1f}%)")
    print(f"   ❌ Failed:     {failed} ({failed/total*100:.1f}%)")
    print(f"   ⏭️  Skipped:    {skipped} ({skipped/total*100:.1f}%)")
    
    # By category
    print(f"\n📂 By category:")
    category_stats = results_df.groupby(['category', 'status']).size().unstack(fill_value=0)
    print(category_stats.to_string())
    
    # Timing stats (for successful files)
    success_df = results_df[results_df['status'] == 'success']
    if not success_df.empty:
        print(f"\n⏱️  Timing (successful files):")
        print(f"   Average: {success_df['time_seconds'].mean():.1f}s per file")
        print(f"   Fastest: {success_df['time_seconds'].min():.1f}s")
        print(f"   Slowest: {success_df['time_seconds'].max():.1f}s")
        print(f"   Total:   {success_df['time_seconds'].sum()/60:.1f} minutes")
    
    # Failed files detail
    if failed > 0:
        print(f"\n❌ Failed files:")
        failed_df = results_df[results_df['status'] == 'failed']
        for idx, row in failed_df.iterrows():
            print(f"   • {row['source_file']}: {row['error']}")
    
    print("\n" + "="*80)
    
    # Display full results table
    print("\n📋 Full results:")
    display(results_df)
    
else:
    print("⚠️ No results to display. Check Step 5.")

## 💾 Step 7: Save Results to CSV

Save processing results for future reference:

In [ ]:
if not results_df.empty and SAVE_RESULTS:
    # Create output directory
    output_path = Path(OUTPUT_DIR) / EVENT_NAME
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Generate filename with timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    results_filename = f"processing_results_{timestamp}.csv"
    results_path = output_path / results_filename
    
    # Save results
    results_df.to_csv(results_path, index=False)
    
    print("💾 RESULTS SAVED")
    print("="*80)
    print(f"\n✅ Saved results to: {results_path.absolute()}")
    print(f"\n📊 Summary:")
    print(f"   Records saved: {len(results_df)}")
    print(f"   Successful:    {len(results_df[results_df['status'] == 'success'])}")
    print(f"   Failed:        {len(results_df[results_df['status'] == 'failed'])}")
    print(f"   Skipped:       {len(results_df[results_df['status'] == 'skipped'])}")
    
elif results_df.empty:
    print("⚠️ No results to save. Check Step 5.")
else:
    print("ℹ️  Results saving disabled (SAVE_RESULTS = False)")

## 💡 Next Steps

After processing:

1. **Review results** - Check the statistics and any failed files
2. **Verify outputs** - Spot-check processed COGs in S3
3. **Handle failures** - Investigate and retry any failed files
4. **Update metadata** - Add metadata to processed files if needed

## 🔧 Troubleshooting

### Common Issues:

1. **"CSV file not found"**
   - Run `renaming_file_template.ipynb` first
   - Check `EVENT_NAME` matches the CSV filename
   - Verify `CSV_DIR` path is correct

2. **"Connection failed"**
   - Check AWS credentials
   - Verify external ID is configured (if using upload role)
   - Test with `test_upload.py`

3. **"All files skipped"**
   - Files already exist in destination
   - Set `OVERWRITE = True` to replace existing files

4. **"Processing failures"**
   - Check source files are valid GeoTIFFs
   - Verify enough disk space for temp files
   - Check S3 write permissions
   - Review error messages in results

5. **"Slow processing"**
   - Large files take longer to process
   - Consider processing in smaller batches
   - Use filters in Step 3 to process by size/category

## 📝 Notes

- Processing time varies based on file size (typically 30s-5min per file)
- COGs are created with zstd compression, level 9
- Nodata values are auto-detected unless specified
- All COGs include 5 overview levels for better performance